<a href="https://colab.research.google.com/github/yujungha/AI-Study/blob/main/10_1%EC%A3%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle competitions download -c titanic

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 22.3MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 22.3MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 2.78MB/s]


In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torch.nn as nn

In [44]:
df = pd.read_csv('train.csv')
df2 = df[['Survived', 'Pclass', 'Sex', 'Age', 'Fare']].reset_index()

df2.loc[:,'Sex'] = df2.loc[:,'Sex'].str.replace('female', '0')
df2.loc[:,'Sex'] = df2.loc[:,'Sex'].str.replace('male', '1')
df2['Sex'] = df2['Sex'].astype(int)
df2['Pclass'] = round(df2['Pclass'] / 3, 1)
df2['Age'] = round(df2['Age'] / 80 , 1)
df2 = df2.dropna(subset=['Age'])
df2['Fare'] = round(df2['Fare'] / df2['Fare'].max() , 3)
df3 = df2['Survived'].values
df3 = df3.reshape(-1, 1)
del df2['index']
del df2['Survived']

x_train = torch.FloatTensor(df2.values)
y_train = torch.FloatTensor(df3)

model = nn.Sequential(
    nn.Linear(4, 50),
    nn.ReLU(),
    nn.Linear(50, 50),
    nn.ReLU(),
    nn.Linear(50, 50),
    nn.ReLU(),
    nn.Linear(50, 50),
    nn.ReLU(),
    nn.Linear(50, 50),
    nn.ReLU(),
    nn.Linear(50, 1)
)

learning_rate = 0.01
epochs = 10000

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
cost = nn.MSELoss()

for i in range(epochs + 1):
  y = model(x_train)
  loss = cost(y, y_train)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if i % 1000 == 0:
    output = torch.abs(torch.round(model(x_train)))
    print(i, loss.item(), '%.2f%%'%((y_train == output).float().mean().numpy() * 100))

0 0.41714999079704285 59.38%
1000 0.09585504978895187 87.25%
2000 0.08263513445854187 89.08%
3000 0.07841148972511292 89.50%
4000 0.07413792610168457 90.34%
5000 0.06868541985750198 90.06%
6000 0.09041319787502289 89.08%
7000 0.06422396004199982 91.04%
8000 0.07793856412172318 90.20%
9000 0.06579536199569702 91.18%
10000 0.06282727420330048 91.88%


In [13]:
test_df = pd.read_csv('test.csv')
test_df = test_df[['Pclass', 'Sex', 'Age', 'Fare']].reset_index()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   index   418 non-null    int64  
 1   Pclass  418 non-null    int64  
 2   Sex     418 non-null    object 
 3   Age     332 non-null    float64
 4   Fare    417 non-null    float64
dtypes: float64(2), int64(2), object(1)
memory usage: 16.5+ KB


In [16]:
test_df['Age'].fillna(test_df['Age'].mean(), inplace=True)
test_df['Age'] = round(test_df['Age'] / test_df['Age'].max() , 1)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   index   418 non-null    int64  
 1   Pclass  418 non-null    int64  
 2   Sex     418 non-null    object 
 3   Age     418 non-null    float64
 4   Fare    417 non-null    float64
dtypes: float64(2), int64(2), object(1)
memory usage: 16.5+ KB


In [37]:
test_df = pd.read_csv('test.csv')
test_df = test_df[['Pclass', 'Sex', 'Age', 'Fare']].reset_index()

test_df.loc[:,'Sex'] = test_df.loc[:,'Sex'].str.replace('female', '0').replace('male', '1')
test_df['Sex'] = test_df['Sex'].astype(int)
test_df['Pclass'] = round(test_df['Pclass'] / test_df['Pclass'].max(), 1)
test_df['Age'].fillna(test_df['Age'].mean(), inplace=True)
test_df['Age'] = round(test_df['Age'] / test_df['Age'].max() , 1)
test_df['Fare'].fillna(test_df['Fare'].mean(), inplace=True)
test_df['Fare'] = round(test_df['Fare'] / test_df['Fare'].max() , 3)
del test_df['index']
test_df

,Pclass,Sex,Age,Fare
0,1.0,1,0.5,0.015
1,1.0,0,0.6,0.014
2,0.7,1,0.8,0.019
3,1.0,1,0.4,0.017
4,1.0,0,0.3,0.024
...,...,...,...,...
413,1.0,1,0.4,0.016
414,0.3,0,0.5,0.213
415,1.0,1,0.5,0.014
416,1.0,1,0.4,0.016


In [45]:
test_df = pd.read_csv('test.csv')
test_df = test_df[['Pclass', 'Sex', 'Age', 'Fare']].reset_index()

test_df.loc[:,'Sex'] = test_df.loc[:,'Sex'].str.replace('female', '0').replace('male', '1')
test_df['Sex'] = test_df['Sex'].astype(int)
test_df['Pclass'] = round(test_df['Pclass'] / test_df['Pclass'].max(), 1)
test_df['Age'].fillna(test_df['Age'].mean(), inplace=True)
test_df['Age'] = round(test_df['Age'] / test_df['Age'].max() , 1)
test_df['Fare'].fillna(test_df['Fare'].mean(), inplace=True)
test_df['Fare'] = round(test_df['Fare'] / test_df['Fare'].max() , 3)
del test_df['index']

x_test = torch.FloatTensor(test_df[['Pclass', 'Sex', 'Age', 'Fare']].values)

In [46]:
submit_df = pd.read_csv('gender_submission.csv')
submit_df['Survived'] = model(x_test).round().int().numpy()
submit_df

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [48]:
submit_df.to_csv('submission.csv', index=False)

In [49]:
!kaggle competitions submit -c titanic -f submission.csv -m "Message"

100% 2.77k/2.77k [00:00<00:00, 5.02kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster

In [10]:
test_df = pd.read_csv('test.csv')
test_df2 = test_df[['Name', 'Age']].reset_index()
test_df2[test_df2['Name'].str.contains('Mr\.')]

,index,Name,Age
0,0,"Kelly, Mr. James",34.5
2,2,"Myles, Mr. Thomas Francis",62.0
3,3,"Wirz, Mr. Albert",27.0
5,5,"Svensson, Mr. Johan Cervin",14.0
7,7,"Caldwell, Mr. Albert Francis",26.0
...,...,...,...
406,406,"Ware, Mr. William Jeffery",23.0
407,407,"Widener, Mr. George Dunton",50.0
413,413,"Spector, Mr. Woolf",NaN
415,415,"Saether, Mr. Simon Sivertsen",38.5
